# Überblick 

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import keras_tuner as kt

from numpy import array
from numpy import hstack
from numpy import empty
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from os import chdir
from os import getcwd
from os import listdir
import math

csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Netherlands'
chdir(csv_file_path)

Y_all = pd.read_csv('heads.csv',decimal='.',index_col=0, delimiter=',', header=0,parse_dates=True)
X_all = pd.read_csv('input_data.csv',decimal='.', delimiter=',',index_col=0, header=0,parse_dates=True)

Y_temp = Y_all['2000-01-01' : '2015-09-10']
X = X_all['2000-01-01' : '2015-09-10']
print(X.shape, Y_temp.shape)

# interpolate
print(X[~(X.index.isin(Y_temp.index))].shape)
Y = Y_temp.reindex(pd.date_range(start=Y_temp.index.min(),end=Y_temp.index.max(),freq='1D')) 
Y.interpolate(method='linear', inplace=True)  
print(X.shape, Y.shape)
print(Y.isna().sum().sum())

(5732, 9) (5696, 1)
(36, 9)
(5732, 9) (5732, 1)
0


In [2]:
X_train = X[:'2011-12-31']
X_test = X['2014-01-01':]
X_valid = X['2012-01-01':'2013-12-31']

Y_train = Y[:'2011-12-31']
Y_test = Y['2014-01-01':]
Y_valid = Y['2012-01-01':'2013-12-31']


csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\data'
chdir(csv_file_path)
from helper import *

(dataset_train,dataset_test,dataset_valid,scaler_X,scaler_Y)=prepare_data(X_train,Y_train,X_valid,Y_valid,X_test,Y_test)

x_final_unsplit = scaler_X.transform(X)
y_final_unsplit = scaler_Y.transform(Y)
dataset_train = np.concatenate((x_final_unsplit,y_final_unsplit), axis=1)
 
iters=3
n_steps_in=30
n_steps_out=1

x_final, y_final = split_sequences_y1(dataset_train, n_steps_in, n_steps_out)

In [3]:
import keras_tuner as kt
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
class Dropout(keras.layers.Dropout):
    def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
        super(Dropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training

        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

In [4]:
test_runs = 100
    
#load models 
csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge'
chdir(csv_file_path)

transformer_model = keras.models.load_model('TF_Netherlands_tune.h5')
transformer_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 9)]      0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 30, 9)        18          input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 30, 9)        2427        layer_normalization[0][0]        
                                                                 layer_normalization[0][0]        
__________________________________________________________________________________________________
dropout (Dropout)               (None, 30, 9)        0           multi_head_attention[0][0]   

## Prediction Transformer

In [5]:
print(x_final.shape)
Y_pred_test_tr = np.empty((x_final.shape[0],n_steps_out,test_runs))
y_correct_utf = scaler_Y.inverse_transform(y_final)#test
for j in range(test_runs): 
    Y_pred_test_tr[:,:,j]=transformer_model.predict(x_final)
    yhat_utf = scaler_Y.inverse_transform(Y_pred_test_tr[:,:,j])#test


(5703, 30, 9)


In [6]:
print(Y_pred_test_tr.shape)
Y_pred = np.mean(Y_pred_test_tr, axis=2)
Y_std = np.std(Y_pred_test_tr, axis=2)
print(Y_std)
Y_pred_test_tr.shape

(5703, 1, 100)
[[0.04015109]
 [0.03817833]
 [0.03646655]
 ...
 [0.03814564]
 [0.03475673]
 [0.0372969 ]]


(5703, 1, 100)

## POC

In [7]:
zT = 13.5
T_ival = [Y_pred - zT*Y_std,Y_pred + zT*Y_std]
def calc_POC(list_of_bounds, measured):
    temp = 0 
    for i in range(len(measured)):
        if measured[i]< list_of_bounds[1][i] and measured[i]> list_of_bounds[0][i]:
            temp = temp + 1
    POC = temp/len(measured)
    return POC

print(T_ival[1].shape)
POC_tr = calc_POC(T_ival,y_final)
print(POC_tr)

(5703, 1)
0.9938628791863932


In [8]:
print(Y_pred_test_tr.shape)
Y_pred_test_tr_flat = Y_pred_test_tr.reshape((test_runs*Y_pred_test_tr.shape[0], 1))
Y_pred_test_utf_flat = scaler_Y.inverse_transform(Y_pred_test_tr_flat)
Y_pred_test_utf = Y_pred_test_utf_flat.reshape((Y_pred_test_tr.shape[0], 1,test_runs))
Y_pred_calib = np.mean(Y_pred_test_utf, axis=2)
Y_std_calib = np.std(Y_pred_test_utf, axis=2)
T_ival_calib = [Y_pred_calib - zT*Y_std_calib,Y_pred_calib + zT*Y_std_calib]
T_ival_calib[1]

(5703, 1, 100)


array([[11.75403629],
       [11.73346916],
       [11.71640189],
       ...,
       [11.7726963 ],
       [11.72625692],
       [11.75186017]])

In [9]:
csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Netherlands'
chdir(csv_file_path)

X_all = pd.read_csv('input_data.csv',decimal='.',index_col='time', delimiter=',', header=0,parse_dates=True)

X_finaltest = X_all['2016-01-01':]
X_finaltest

,rr,tg,tn,tx,pp,hu,fg,qq,et
time,,,,,,,,,
2016-01-01,0.0,4.700000,1.42,8.44,1022.70000,91.650000,2.96,22.0,0.232114
2016-01-02,2.3,3.050000,1.81,4.12,1012.00000,89.743750,6.32,13.0,0.130259
2016-01-03,4.2,0.800000,-0.52,1.96,1002.00000,93.418180,6.54,14.0,0.129784
2016-01-04,4.3,-2.590000,-3.43,0.99,990.80000,91.125000,5.77,12.0,0.097585
2016-01-05,0.0,-2.180000,-3.49,-1.25,994.20000,92.715385,5.16,10.0,0.082576
...,...,...,...,...,...,...,...,...,...
2021-12-27,0.0,2.110000,-0.07,5.08,998.70000,92.220000,3.13,20.0,0.195135
2021-12-28,5.5,6.610000,3.88,7.97,990.40000,93.700005,3.36,23.0,0.262438
2021-12-29,4.0,7.030000,5.49,10.78,1001.30000,93.700005,2.99,21.0,0.241433


In [10]:
X_finaltest_trf = scaler_X.transform(X_finaltest)
X_ftest = split_sequences_test(X_finaltest_trf,n_steps_in, n_steps_out)
X_ftest.shape

(2163, 30, 9)

In [11]:
print(x_final.shape)
Y_pred_ftest_tr = np.empty((X_ftest.shape[0],n_steps_out,test_runs))
for j in range(test_runs): 
    Y_pred_ftest_tr[:,:,j]=transformer_model.predict(X_ftest)

(5703, 30, 9)


In [12]:
Y_pred_ftest_tr_flat = Y_pred_ftest_tr.reshape((100*Y_pred_ftest_tr.shape[0], 1))
Y_pred_ftest_utf_flat = scaler_Y.inverse_transform(Y_pred_ftest_tr_flat)
Y_pred_ftest_utf = Y_pred_ftest_utf_flat.reshape((Y_pred_ftest_tr.shape[0], 1,100))

yhat_mean = np.mean(Y_pred_ftest_utf, axis=2)
yhat_std = np.std(Y_pred_ftest_utf, axis=2)
T_ival_utf = [yhat_mean - zT*yhat_std,yhat_mean + zT*yhat_std]
yhat_mean.shape

(2163, 1)

In [13]:
final_idx_calib = pd.date_range(start='2000-01-30', end='2015-09-10')
final_idx_test = pd.date_range(start='2016-01-30', end='2021-12-31')
final_df_calib = pd.DataFrame(index=final_idx_calib)
final_df_test = pd.DataFrame(index=final_idx_test)
final_df_test.shape

(2163, 0)

In [14]:
print(Y_pred.shape)
final_df_calib['head forecast'] = Y_pred_calib
final_df_calib['95% lower'] = T_ival_calib[0]
final_df_calib['95% upper'] = T_ival_calib[1]
final_df_calib

(5703, 1)


,head forecast,95% lower,95% upper
2000-01-30,11.314984,10.875932,11.754036
2000-01-31,11.315989,10.898509,11.733469
2000-02-01,11.317640,10.918878,11.716402
2000-02-02,11.309486,10.920897,11.698075
2000-02-03,11.307523,10.870785,11.744260
...,...,...,...
2015-09-06,11.343170,10.935135,11.751206
2015-09-07,11.346622,10.925023,11.768220
2015-09-08,11.355574,10.938451,11.772696
2015-09-09,11.346192,10.966127,11.726257


In [15]:
final_df_test['head forecast'] = yhat_mean
final_df_test['95% lower'] = T_ival_utf[0]
final_df_test['95% upper'] = T_ival_utf[1]
final_df_test

,head forecast,95% lower,95% upper
2016-01-30,11.319615,10.914517,11.724714
2016-01-31,11.315342,10.906989,11.723695
2016-02-01,11.311375,10.886043,11.736706
2016-02-02,11.309068,10.884096,11.734040
2016-02-03,11.304267,10.911088,11.697446
...,...,...,...
2021-12-27,11.264746,10.896912,11.632580
2021-12-28,11.263098,10.901156,11.625039
2021-12-29,11.264554,10.820980,11.708127
2021-12-30,11.269738,10.923241,11.616234


In [16]:
csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/submissions'
chdir(csv_file_path)
final_df_calib.to_csv('Netherlands_calib.csv', float_format='%.2f')
final_df_test.to_csv('Netherlands_test.csv', float_format='%.2f')